В цьому домашньому завданні ми знову працюємо з даними з нашого змагання ["Bank Customer Churn Prediction (DLU Course)"](https://www.kaggle.com/t/7c080c5d8ec64364a93cf4e8f880b6a0).

Тут ми побудуємо рішення задачі класифікації з використанням алгоритмів бустингу: XGBoost та LightGBM, а також використаємо бібліотеку HyperOpt для оптимізації гіперпараметрів.

0. Зчитайте дані `train.csv` в змінну `raw_df` та скористайтесь наведеним кодом нижче аби розділити дані на трнувальні та валідаційні і розділити дані на ознаки з матириці Х та цільову змінну. Назви змінних `train_inputs, train_targets, train_inputs, train_targets` можна змінити на ті, які Вам зручно.

  Наведений скрипт - частина отриманого мною скрипта для обробки даних. Ми тут не викнуємо масштабування та обробку категоріальних змінних, бо хочемо це делегувати алгоритмам, які будемо використовувати. Якщо щось не розумієте в наведених скриптах, рекомендую розібратись: навичка читати код - важлива складова роботи в машинному навчанні.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Add the directory to the Python path
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks')

# Зчитування даних з файлу train.csv
raw_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/train.csv")

# Припустимо, що остання колонка є цільовою змінною
X = raw_df.iloc[:, :-1]  # Всі колонки, крім останньої
y = raw_df.iloc[:, -1]   # Остання колонка

# Розділення на тренувальні та валідаційні набори
train_inputs, val_inputs, train_targets, val_targets = train_test_split(X, y, test_size=0.2, random_state=42)

# Виведення розмірів отриманих наборів
print(f'Train inputs shape: {train_inputs.shape}')
print(f'Train targets shape: {train_targets.shape}')
print(f'Validation inputs shape: {val_inputs.shape}')
print(f'Validation targets shape: {val_targets.shape}')

Mounted at /content/drive
Train inputs shape: (12000, 13)
Train targets shape: (12000,)
Validation inputs shape: (3000, 13)
Validation targets shape: (3000,)


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from typing import Tuple, Dict, Any


def split_train_val(df: pd.DataFrame, target_col: str, test_size: float = 0.2, random_state: int = 42) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Split the dataframe into training and validation sets.

    Args:
        df (pd.DataFrame): The raw dataframe.
        target_col (str): The target column for stratification.
        test_size (float): The proportion of the dataset to include in the validation split.
        random_state (int): Random state for reproducibility.

    Returns:
        Tuple[pd.DataFrame, pd.DataFrame]: Training and validation dataframes.
    """
    train_df, val_df = train_test_split(df, test_size=test_size, random_state=random_state, stratify=df[target_col])
    return train_df, val_df


def separate_inputs_targets(df: pd.DataFrame, input_cols: list, target_col: str) -> Tuple[pd.DataFrame, pd.Series]:
    """
    Separate inputs and targets from the dataframe.

    Args:
        df (pd.DataFrame): The dataframe.
        input_cols (list): List of input columns.
        target_col (str): Target column.

    Returns:
        Tuple[pd.DataFrame, pd.Series]: DataFrame of inputs and Series of targets.
    """
    inputs = df[input_cols].copy()
    targets = df[target_col].copy()
    return inputs, targets

1. В тренувальному та валідаційному наборі перетворіть категоріальні ознаки на тип `category`. Можна це зробити двома способами:
 1. `df[col_name].astype('category')`, як було продемонстровано в лекції
 2. використовуючи метод `pd.Categorical(df[col_name])`

In [3]:
# Припустимо, що категоріальні ознаки визначені в цьому списку
categorical_columns = ['Geography', 'Gender']  # Замініть на ваші категоріальні колонки

# Метод 1: Використання astype('category')
for col in categorical_columns:
    train_inputs[col] = train_inputs[col].astype('category')
    val_inputs[col] = val_inputs[col].astype('category')

# Метод 2: Використання pd.Categorical
for col in categorical_columns:
    train_inputs[col] = pd.Categorical(train_inputs[col])
    val_inputs[col] = pd.Categorical(val_inputs[col])

# Виведення інформації про типи даних для перевірки
print(train_inputs.dtypes)
print(val_inputs.dtypes)

id                    int64
CustomerId          float64
Surname              object
CreditScore         float64
Geography          category
Gender             category
Age                 float64
Tenure              float64
Balance             float64
NumOfProducts       float64
HasCrCard           float64
IsActiveMember      float64
EstimatedSalary     float64
dtype: object
id                    int64
CustomerId          float64
Surname              object
CreditScore         float64
Geography          category
Gender             category
Age                 float64
Tenure              float64
Balance             float64
NumOfProducts       float64
HasCrCard           float64
IsActiveMember      float64
EstimatedSalary     float64
dtype: object


2. Навчіть на отриманих даних модель `XGBoostClassifier`. Параметри алгоритму встановіть на свій розсуд, ми далі будемо їх тюнити. Рекомендую тренувати не дуже складну модель.

  Опис всіх конфігураційних параметрів XGBoostClassifier - тут https://xgboost.readthedocs.io/en/stable/parameter.html#global-config

  **Важливо:** зробіть такі налаштування `XGBoostClassifier` аби він самостійно обробляв незаповнені значення в даних і обробляв категоріальні колонки.

  Можна також, якщо працюєте в Google Colab, увімкнути можливість використання GPU (`Runtime -> Change runtime type -> T4 GPU`) і встановити параметр `device='cuda'` в `XGBoostClassifier` для пришвидшення тренування бустинг моделі.
  
  Після тренування моделі
  1. Виміряйте точність з допомогою AUROC на тренувальному та валідаційному наборах.
  2. Зробіть висновок про отриману модель: вона хороша/погана, чи є high bias/high variance?
  3. Порівняйте якість цієї моделі з тою, що ви отрмали з використанням DecisionTrees раніше. Чи вийшло покращити якість?

In [4]:
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

# Перетворення категоріальних ознак на тип category
for col in train_inputs.select_dtypes(include=['object', 'category']).columns:
    train_inputs[col] = train_inputs[col].astype('category')
    val_inputs[col] = val_inputs[col].astype('category')

# Ініціалізація та тренування XGBoostClassifier
model = XGBClassifier(
    use_label_encoder=False,  # Додатковий параметр для уникнення попередження
    eval_metric='auc',        # Метрика для валідації
    tree_method='gpu_hist',   # Використання GPU
    enable_categorical=True   # Обробка категоріальних колонок
)

# Тренування моделі
model.fit(train_inputs, train_targets)

# Прогнозування на тренувальних та валідаційних наборах
train_preds = model.predict_proba(train_inputs)[:, 1]
val_preds = model.predict_proba(val_inputs)[:, 1]

# Вимірювання точності з допомогою AUROC
train_auc = roc_auc_score(train_targets, train_preds)
val_auc = roc_auc_score(val_targets, val_preds)

# Виведення результатів
print(f'Train AUROC: {train_auc}')
print(f'Validation AUROC: {val_auc}')

# Висновки
if val_auc > 0.7:
    print("Модель є хорошою.")
else:
    print("Модель потребує покращення.")

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [22:28:27] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [22:28:27] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Train AUROC: 0.9996275383083292
Validation AUROC: 0.9138309421138706
Модель є хорошою.


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [22:28:28] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [22:28:28] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


Висновки

Train AUROC 0.9996 - дуже високий показник, майже досягає максимальної можливої точності. Це свідчить про те, що модель дуже добре навчається на тренувальних даних.
Validation AUROC 0.9138 - також дуже високий показник, що свідчить про те, що модель добре справляється з новими даними, які вона не бачила під час навчання.

Модель є хорошою і значно перевищує поріг 0.7, що вказує на високу точність та гарну узагальненість моделі на нових даних.

High Bias - малоймовірно, оскільки модель показує дуже високу точність як на тренувальних, так і на валідаційних даних.

High Variance - є невеликий ризик, оскільки є невеликий розрив між Train AUROC і Validation AUROC, але він не критичний. Різниця в AUROC вказує на те, що модель добре узагальнює, хоча може мати деякі елементи перенавчання.

В моєму випадку результати значно кращі за DecisionTrees

3. Використовуючи бібліотеку `Hyperopt` і приклад пошуку гіперпараметрів для `XGBoostClassifier` з лекції знайдіть оптимальні значення гіперпараметрів `XGBoostClassifier` для нашої задачі. Задайте свою сітку гіперпараметрів виходячи з тих параметрів, які ви б хотіли перебрати. Поставте кількість раундів в підборі гіперпараметрів рівну **20**.

  **Увага!** Для того, аби скористатись hyperopt, нам треба задати функцію `objective`. В ній ми маємо задати loss - це може будь-яка метрика, але бажано використовувтаи ту, яка цільова в вашій задачі. Чим менший лосс - тим ліпша модель на думку hyperopt. Тож, тут нам треба задати loss - негативне значення AUROC. В лекції ми натомість використовували Accuracy.

  Після успішного завершення пошуку оптимальних гіперпараметрів
    - виведіть найкращі значення гіперпараметрів
    - створіть в окремій зміній `final_clf` модель `XGBoostClassifier` з найкращими гіперпараметрами
    - навчіть модель `final_clf`
    - оцініть якість моделі `final_clf` на тренувальній і валідаційній вибірках з допомогою AUROC.
    - зробіть висновок про якість моделі. Чи стала вона краще порівняно з попереднім пунктом (2) цього завдання?

In [5]:
!pip install hyperopt xgboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 MB 1.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.3.1+cu121 requires nvidia-cublas-cu12==12.1.3.1; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.1+cu121 requires nvidia-cuda-cupti-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.1+cu121 requires nvidia-cuda-nvrtc-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.1+cu121 requires nvidia-cuda-runtime-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.1+cu121 requires nvidia-cudnn-cu12==8.9.2.26; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.1+cu121 requ

In [6]:
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from hyperopt.pyll.base import scope

In [7]:
def objective(params):
    model = XGBClassifier(
        use_label_encoder=False,
        eval_metric='auc',
        tree_method='gpu_hist',  # Використання GPU
        enable_categorical=True,
        **params
    )
    model.fit(train_inputs, train_targets)
    val_preds = model.predict_proba(val_inputs)[:, 1]
    auc = roc_auc_score(val_targets, val_preds)
    return {'loss': -auc, 'status': STATUS_OK}

In [8]:
space = {
    'n_estimators': scope.int(hp.quniform('n_estimators', 50, 500, 50)),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.3),
    'max_depth': scope.int(hp.quniform('max_depth', 3, 10, 1)),
    'min_child_weight': scope.int(hp.quniform('min_child_weight', 1, 10, 1)),
    'gamma': hp.uniform('gamma', 0, 0.5),
    'subsample': hp.uniform('subsample', 0.6, 1.0),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.6, 1.0),
    'reg_alpha': hp.uniform('reg_alpha', 0, 0.5),
    'reg_lambda': hp.uniform('reg_lambda', 0.5, 1.0)
}

trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=20,
            trials=trials)

  0%|          | 0/20 [00:00<?, ?trial/s, best loss=?]

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [22:28:49] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [22:28:49] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



  5%|▌         | 1/20 [00:00<00:16,  1.18trial/s, best loss: -0.9213938427970729]

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [22:28:50] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [22:28:50] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [22:28:50] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



 10%|█         | 2/20 [00:02<00:25,  1.40s/trial, best loss: -0.9213938427970729]

...

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [22:29:12] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [22:29:12] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [22:29:12] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



 95%|█████████▌| 19/20 [00:24<00:01,  1.09s/trial, best loss: -0.9278034329762318]

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [22:29:13] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [22:29:13] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [22:29:13] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



100%|██████████| 20/20 [00:24<00:00,  1.22s/trial, best loss: -0.9278034329762318]


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [22:29:13] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)



In [9]:
print("Best hyperparameters: ", best)

Best hyperparameters:  {'colsample_bytree': 0.7151961620121172, 'gamma': 0.22026075030455833, 'learning_rate': 0.021680278575909723, 'max_depth': 3.0, 'min_child_weight': 6.0, 'n_estimators': 400.0, 'reg_alpha': 0.48629392389397114, 'reg_lambda': 0.8633236547017025, 'subsample': 0.7269753440879103}


In [10]:
final_clf = XGBClassifier(
    use_label_encoder=False,
    eval_metric='auc',
    tree_method='gpu_hist',  # Використання GPU
    enable_categorical=True,
    n_estimators=int(best['n_estimators']),
    learning_rate=best['learning_rate'],
    max_depth=int(best['max_depth']),
    min_child_weight=int(best['min_child_weight']),
    gamma=best['gamma'],
    subsample=best['subsample'],
    colsample_bytree=best['colsample_bytree'],
    reg_alpha=best['reg_alpha'],
    reg_lambda=best['reg_lambda']
)

final_clf.fit(train_inputs, train_targets)

# Прогнозування на тренувальних та валідаційних наборах
train_preds_final = final_clf.predict_proba(train_inputs)[:, 1]
val_preds_final = final_clf.predict_proba(val_inputs)[:, 1]

# Вимірювання точності з допомогою AUROC
train_auc_final = roc_auc_score(train_targets, train_preds_final)
val_auc_final = roc_auc_score(val_targets, val_preds_final)

# Виведення результатів
print(f'Final Train AUROC: {train_auc_final}')
print(f'Final Validation AUROC: {val_auc_final}')

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [22:29:13] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [22:29:13] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Final Train AUROC: 0.9445945888293124
Final Validation AUROC: 0.9278034329762318


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [22:29:14] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


Train AUROC

Зниження - після тюнінгу значення AUROC на тренувальній вибірці знизилось з 0.9996 до 0.9534. Це може свідчити про те, що модель стала менш схильною до перенавчання (overfitting).

Висновок. Початково модель мала дуже високу точність на тренувальних даних, що може свідчити про можливе перенавчання. Після тюнінгу цей показник зменшився, що може означати кращу узагальненість моделі.

Validation AUROC

Покращення - значення AUROC на валідаційній вибірці зросло з 0.9138 до 0.9273. Це свідчить про те, що модель краще узагальнює на невидимих даних.

Висновок. Підвищення показника AUROC на валідаційних даних означає, що тюнінг гіперпараметрів допоміг поліпшити якість моделі і її здатність до узагальнення.

Загальний висновок
Після тюнінгу гіперпараметрів ми отримали кращий результат на валідаційних даних, що є основним показником якості моделі.Підтвердження покращення валідаційного AUROC при деякому зниженні тренувального AUROC вказує на зменшення перенавчання.
Таким чином, можна зробити висновок, що тюнінг гіперпараметрів допоміг покращити якість моделі, зробивши її менш схильною до перенавчання та кращою у здатності узагальнювати на нових даних.

4. Навчіть на наших даних модель LightGBM. Параметри алгоритму встановіть на свій розсуд, ми далі будемо їх тюнити. Рекомендую тренувати не дуже складну модель.

  Опис всіх конфігураційних параметрів LightGBM - тут https://lightgbm.readthedocs.io/en/latest/Parameters.html

  **Важливо:** зробіть такі налаштування LightGBM аби він самостійно обробляв незаповнені значення в даних і обробляв категоріальні колонки.

  Аби передати категоріальні колонки в LightGBM - необхідно виявити їх індекси і передати в параметрі `cat_feature=cat_feature_indexes`

  Після тренування моделі
  1. Виміряйте точність з допомогою AUROC на тренувальному та валідаційному наборах.
  2. Зробіть висновок про отриману модель: вона хороша/погана, чи є high bias/high variance?
  3. Порівняйте якість цієї моделі з тою, що ви отрмали з використанням XGBoostClassifier раніше. Чи вийшло покращити якість?

In [11]:
!pip install lightgbm
import lightgbm as lgb

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [12]:
# Індекси категоріальних колонок для LightGBM
cat_feature_indexes = [train_inputs.columns.get_loc(col) for col in train_inputs.select_dtypes(include=['category']).columns]

In [13]:
# Налаштування параметрів LightGBM
params = {
    'objective': 'binary',
    'metric': 'auc',
    'boosting_type': 'gbdt',
    'learning_rate': 0.1,
    'num_leaves': 31,
    'max_depth': -1,  # No limit
    'categorical_feature': cat_feature_indexes,
    'verbose': -1
}

# Створення LightGBM Dataset
train_data = lgb.Dataset(train_inputs, label=train_targets, categorical_feature=cat_feature_indexes)
val_data = lgb.Dataset(val_inputs, label=val_targets, categorical_feature=cat_feature_indexes, reference=train_data)

# Навчання моделі з ранньою зупинкою за допомогою callbacks
model = lgb.train(
    params,
    train_data,
    valid_sets=[val_data],
    valid_names=['valid'],
    num_boost_round=1000,
    callbacks=[lgb.early_stopping(stopping_rounds=50)]  # Використання ранньої зупинки
)

/usr/local/lib/python3.10/dist-packages/lightgbm/basic.py:2108: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(
/usr/local/lib/python3.10/dist-packages/lightgbm/basic.py:2108: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(


Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[23]	valid's auc: 0.920091


In [14]:
# Прогнозування на тренувальних та валідаційних наборах
train_preds = model.predict(train_inputs)
val_preds = model.predict(val_inputs)

# Вимірювання точності з допомогою AUROC
train_auc = roc_auc_score(train_targets, train_preds)
val_auc = roc_auc_score(val_targets, val_preds)

# Виведення результатів
print(f'Train AUROC: {train_auc}')
print(f'Validation AUROC: {val_auc}')

Train AUROC: 0.9586274698319173
Validation AUROC: 0.9200910042451214


Train AUROC - модель XGBoost демонструє вищий AUROC на тренувальних даних, що вказує на можливе переобладнання (overfitting). Високий тренувальний AUROC і дуже низький валідаційний AUROC у XGBoost вказують на те, що модель XGBoost може мати високий variance.

Validation AUROC - LightGBM показує трохи кращий результат на валідаційних даних порівняно з XGBoost. Це свідчить про те, що LightGBM краще узагальнює на нових даних і менш схильний до переобладнання порівняно з XGBoost.

Висновок
Модель LightGBM має високий AUROC на тренувальних та валідаційних даних, що свідчить про хорошу продуктивність і помірний рівень переобладнання.
В порівнянні з XGBoostClassifier, LightGBM має кращі результати на валідаційних даних. Це свідчить про те, що LightGBM краще узагальнює на нових даних і може бути кращим вибором для цієї задачі.
Загалом, LightGBM забезпечує кращу якість моделі на валідаційних даних порівняно з XGBoost, що робить його кращим вибором для цієї конкретної задачі.

5. Використовуючи бібліотеку `Hyperopt` і приклад пошуку гіперпараметрів для `LightGBM` з лекції знайдіть оптимальні значення гіперпараметрів `LightGBM` для нашої задачі. Задайте свою сітку гіперпараметрів виходячи з тих параметрів, які ви б хотіли перебрати. Поставте кількість раундів в підборі гіперпараметрів рівну **10**.

  **Увага!** Для того, аби скористатись hyperopt, нам треба задати функцію `objective`. І тут ми також ставимо loss - негативне значення AUROC, як і при пошуці гіперпараметрів для XGBoost. До речі, можна спробувати написати код так, аби в objective передавати лише модель і не писати схожий код двічі :)

  Після успішного завершення пошуку оптимальних гіперпараметрів
    - виведіть найкращі значення гіперпараметрів
    - створіть в окремій зміній `final_lgb_clf` модель `LightGBM` з найкращими гіперпараметрами
    - навчіть модель `final_lgb_clf`
    - оцініть якість моделі `final_lgb_clf` на тренувальній і валідаційній вибірках з допомогою AUROC.
    - зробіть висновок про якість моделі. Чи стала вона краще порівняно з попереднім пунктом (4) цього завдання?

In [15]:
!pip install lightgbm hyperopt

In [16]:
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
# Індекси категоріальних колонок для LightGBM
cat_feature_indexes = [train_inputs.columns.get_loc(col) for col in train_inputs.select_dtypes(include=['category']).columns]

# Створення LightGBM Dataset з параметром free_raw_data=False
train_data = lgb.Dataset(train_inputs, label=train_targets, categorical_feature=cat_feature_indexes, free_raw_data=False)
val_data = lgb.Dataset(val_inputs, label=val_targets, categorical_feature=cat_feature_indexes, reference=train_data, free_raw_data=False)

In [17]:
def objective(params):
    # Налаштування параметрів LightGBM
    params = {
        'objective': 'binary',
        'metric': 'auc',
        'boosting_type': 'gbdt',
        'verbose': -1,
        **params
    }

    # Навчання моделі
    model = lgb.train(
        params,
        train_data,
        valid_sets=[val_data],
        valid_names=['valid'],
        num_boost_round=1000,
        callbacks=[lgb.early_stopping(stopping_rounds=50)]
    )

    # Прогнозування на валідаційних даних
    val_preds = model.predict(val_inputs)

    # Вимірювання AUROC
    val_auc = roc_auc_score(val_targets, val_preds)

    # Оптимізація: мінімізуємо негативний AUROC
    return {
        'loss': -val_auc,
        'status': STATUS_OK
    }

In [18]:
space = {
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.2),
    'num_leaves': hp.choice('num_leaves', [31, 63, 127]),  # Зазначити значення більше 1
    'max_depth': hp.choice('max_depth', [-1, 10, 20, 30]),
    'min_child_samples': hp.choice('min_child_samples', [10, 20, 30]),
    'subsample': hp.uniform('subsample', 0.6, 1.0),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.6, 1.0),
    'reg_alpha': hp.uniform('reg_alpha', 0, 1),
    'reg_lambda': hp.uniform('reg_lambda', 0, 1)
}

In [19]:
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK

# Запуск оптимізації Hyperopt
trials = Trials()
best = fmin(
    fn=objective,
    space=space,
    algo=tpe.suggest,
    max_evals=10,  # Кількість раундів
    trials=trials
)

print(f'Best hyperparameters: {best}')

Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[8]	valid's auc: 0.927728
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[23]	valid's auc: 0.92994
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[36]	valid's auc: 0.918994
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[13]	valid's auc: 0.920568
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[65]	valid's auc: 0.920408
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[8]	valid's auc: 0.924208
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[3]	valid's auc: 0.924882
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[14]	valid's auc: 0.927086
Training until valid

In [20]:
# Налаштування найкращих параметрів
best_params = {
    'objective': 'binary',
    'metric': 'auc',
    'boosting_type': 'gbdt',
    'verbose': -1,
    **best
}

# Створення фінальної моделі
final_lgb_clf = lgb.train(
    best_params,
    train_data,
    valid_sets=[val_data],
    valid_names=['valid'],
    num_boost_round=1000,
    callbacks=[lgb.early_stopping(stopping_rounds=50)]
)

# Прогнозування на тренувальних та валідаційних наборах
train_preds = final_lgb_clf.predict(train_inputs)
val_preds = final_lgb_clf.predict(val_inputs)

# Вимірювання AUROC
train_auc = roc_auc_score(train_targets, train_preds)
val_auc = roc_auc_score(val_targets, val_preds)

print(f'Final Train AUROC: {train_auc}')
print(f'Final Validation AUROC: {val_auc}')

Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[565]	valid's auc: 0.917189
Final Train AUROC: 0.9444019369731729
Final Validation AUROC: 0.9171886178981721


Train AUROC 0.9446 – це висока точність на тренувальних даних, що вказує на добре підходження моделі до навчальних даних.

Validation AUROC 0.9171 – це також висока точність на валідаційних даних, що свідчить про хорошу узагальненість моделі.
Порівняння з попередніми моделями:

Хоча LightGBM показала дуже хороші результати, XGBoost мала трохи кращу точність на валідаційних даних. Це може вказувати на те, що модель XGBoost краще підійшла для даних у цьому конкретному випадку.

Модель LightGBM показала дуже хороші результати, але XGBoost залишилася трохи кращою за точністю на валідаційних даних. Однак, обидві моделі демонструють відмінну якість, і обрана модель може залежати від подальших вимог або специфічних характеристик даних.

Якщо точність на валідаційних даних є критично важливою, можна продовжити використовувати XGBoost. Однак, якщо LightGBM демонструє кращу ефективність у термінах швидкості навчання або ресурсів, то вона також є хорошим вибором.

6. Оберіть модель з експериментів в цьому ДЗ і зробіть новий `submission` на Kaggle та додайте код для цього і скріншот скора на публічному лідерборді.
  
  **Напишіть коментар, чому ви обрали саме цю модель?**

  І я вас вітаю - це останнє завдання з цим набором даних 💪 На цьому етапі корисно проаналізувати, які моделі показали себе найкраще і подумати, чому.

In [24]:
import pandas as pd
import lightgbm as lgb

# Завантаження даних
train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/test.csv')
submission = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sample_submission.csv')

# Визначення категоріальних ознак
categorical_features = ['Geography', 'Gender']

# Функція для обробки категоріальних колонок
def preprocess_data(df: pd.DataFrame, categorical_features: list) -> pd.DataFrame:
    df = df.copy()
    for col in categorical_features:
        if col in df.columns:
            df[col] = pd.Categorical(df[col])
    return df

# Обробка тренувальних даних
train_df = preprocess_data(train_df, categorical_features)

# Збереження категорій з тренувальних даних
saved_categories = {col: train_df[col].cat.categories for col in categorical_features}

# Функція для обробки нових даних з урахуванням категорій
def preprocess_new_data(df: pd.DataFrame, categorical_features: list, saved_categories: dict) -> pd.DataFrame:
    df = df.copy()
    for col in categorical_features:
        if col in df.columns:
            df[col] = pd.Categorical(df[col], categories=saved_categories.get(col, []))
            df[col] = df[col].cat.codes
    return df

# Видалення стовпця 'Surname' з тренувальних і тестових даних
train_df = train_df.drop(columns=['Surname'])
test_df = test_df.drop(columns=['Surname'])

# Обробка тренувальних даних для моделі
processed_train = preprocess_new_data(train_df, categorical_features, saved_categories)

# Обробка тестових даних
preprocessed_test = preprocess_new_data(test_df, categorical_features, saved_categories)

# Визначення характеристик і мети
X_train = processed_train.drop(['id', 'Exited'], axis=1)
y_train = processed_train['Exited']

# Створення і тренування моделі
final_lgb_clf = lgb.LGBMClassifier()
final_lgb_clf.fit(X_train, y_train, categorical_feature=categorical_features)

# Передбачення на тестових даних
proba_predictions = final_lgb_clf.predict(preprocessed_test.drop('id', axis=1))

# Додавання прогнозів у DataFrame для submission
submission = submission[['id']].copy()  # Оновлення колонок для submission
submission['Exited'] = proba_predictions

# Перегляд результатів
print(submission.head())

# Збереження submission-файлу
submission.to_csv('/content/drive/MyDrive/Colab Notebooks/submission.csv', index=False)

[LightGBM] [Info] Number of positive: 3052, number of negative: 11948
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001506 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1100
[LightGBM] [Info] Number of data points in the train set: 15000, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.203467 -> initscore=-1.364767
[LightGBM] [Info] Start training from score -1.364767
      id  Exited
0  15000     0.0
1  15001     0.0
2  15002     0.0
3  15003     1.0
4  15004     0.0


На жаль, не встигла вчасно засабмітити:(